In [ ]:
pip install pytorch_lightning

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

np.random.seed(11)

# Данные

Беру информацию об оценках фильмов пользователями ratings.csv из полного датасета MovieLens

In [77]:
ratings = pd.read_csv('newratings.csv', parse_dates=['timestamp'])
ratings = ratings.iloc[:-1]
ratings['movieId'] = ratings['movieId'].apply(int)
ratings['userId']  = ratings['userId'].apply(int)
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


Возьму только 7000 случайных уникальных пользователей.

In [78]:
random_users = ratings['userId'].unique()[:7000]
ratings = ratings.loc[ratings['userId'].isin(random_users)]

Для каждого пользователя в валидационную выборку беру его самую последнюю оценку, а все остальные оценки этого пользователя используем в обучающей выборке. Это позволит избежать заглядывания в будущее. Я не хочу случайно  разбивать данные в рекомендательной системе, так как порядок оценок пользователя важен! Он отражает текущие предпочтения пользователя и изменение интересов  со временем. 


In [80]:
ratings['rank_latest'] = ratings.groupby(['userId'])['timestamp'] \
                                .rank(method='first', ascending=False)

train_ratings = ratings[ratings['rank_latest'] != 1]
test_ratings = ratings[ratings['rank_latest'] == 1]

train_ratings = train_ratings[['userId', 'movieId', 'rating']]
test_ratings = test_ratings[['userId', 'movieId', 'rating']]

# Построение модели

In [82]:
class DataHolder(Dataset):
   
    def __init__(self, ratings, all_movies):
        self.users, self.movies, self.labels = self.get_dataset(ratings, all_movies)

    def get_dataset(self, ratings, all_movies):
        users, movies, labels = [], [], []
        user_movie_set = set(zip(ratings['userId'], ratings['movieId']))

        num_negatives = 3
        for u, i in user_movie_set:
            users.append(u)
            movies.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_movie = np.random.choice(all_movies)
                while (u, negative_movie) in user_movie_set:
                    negative_movie = np.random.choice(all_movies)
                users.append(u)
                movies.append(negative_movie)
                labels.append(0)

        return torch.tensor(users), torch.tensor(movies), torch.tensor(labels)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.movies[idx], self.labels[idx]

In [83]:
class CollaborativeFiltering(pl.LightningModule):
  
    def __init__(self, num_users, num_movies, ratings, all_movies):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_movies, embedding_dim=8)
        self.linear1 = nn.Linear(in_features=16, out_features=64)
        self.linear2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.ratings = ratings
        self.all_movies = all_movies
    
    def forward(self, user_input, item_input):
        
        # пропускаем через слои эмбеддинга
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # склеивание двух слоев эмбеддинга
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # пропускаем через полносвязные слои
        vector = nn.ReLU()(self.linear1(vector))
        vector = nn.ReLU()(self.linear2(vector))

        # выходной слой - вероятность,
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    # для бустинга
    def last_layer(self, user_input, item_input):
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)
        vector = torch.cat([user_embedded, item_embedded], dim=-1)
        vector = nn.ReLU()(self.linear1(vector))
        vector = nn.ReLU()(self.linear2(vector))
        return vector

    def get_embeddings(self, user_input, item_input):
        return  self.user_embedding(user_input), self.item_embedding(item_input)

    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(DataHolder(self.ratings, self.all_movies),
                          batch_size=512, num_workers=4)

In [84]:
# инициализация модели
num_users  = ratings['userId'].max()+1
num_movies = ratings['movieId'].max()+1
all_movies = ratings['movieId'].unique()

model = CollaborativeFiltering(num_users, num_movies, train_ratings, all_movies)

Пришло время обучить модель!

In [ ]:
trainer = pl.Trainer(
    max_epochs=5,
    accelerator='gpu',
    reload_dataloaders_every_n_epochs=1, 
    # создаю новый случайно выбранный набор негативных выборок для каждой эпохи 
    logger=False,
)

trainer.fit(model)


# BOOSTING

попробуем выводить предпоследний слой предобученной модели в бустинг на деревьях для улучшения классификации:

In [153]:
import lightgbm
params = {
    'boosting_type': 'gbdt', 
    'n_estimators': 1000, # 1000 деревьев
    'learning_rate': 0.1,
    'num_leaves': 3, # деревья из 3ех листьев
    'reg_alpha': 10,
    'reg_lambda': 100,
    'subsample_freq': 0, 
    'subsample': 1, 
    'n_jobs': 4, 
    'random_state': 4242 
}
boosting = lightgbm.LGBMClassifier(**params)

In [ ]:
train_data = DataHolder(train_ratings, model.all_movies)
test_data  = DataHolder(test_ratings, model.all_movies)
X_train = model.last_layer(train_data.users, train_data.movies).detach().numpy()
Y_train = train_data.labels.detach().numpy()
X_val = model.last_layer(test_data.users, test_data.movies).detach().numpy()
Y_val = test_data.labels.detach().numpy()

In [156]:
boosting.fit(
    X=X_train,
    y=y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_metric='auc', 
    callbacks=[lightgbm.early_stopping(stopping_rounds=200, first_metric_only=True),
               lightgbm.log_evaluation(50)]
)

Training until validation scores don't improve for 200 rounds
[50]	training's auc: 0.935803	training's binary_logloss: 0.278222	valid_1's auc: 0.921827	valid_1's binary_logloss: 0.305325
[100]	training's auc: 0.938208	training's binary_logloss: 0.271124	valid_1's auc: 0.924477	valid_1's binary_logloss: 0.304674
[150]	training's auc: 0.939239	training's binary_logloss: 0.268683	valid_1's auc: 0.925389	valid_1's binary_logloss: 0.303842
[200]	training's auc: 0.939792	training's binary_logloss: 0.267407	valid_1's auc: 0.92568	valid_1's binary_logloss: 0.304229
[250]	training's auc: 0.94015	training's binary_logloss: 0.266591	valid_1's auc: 0.92586	valid_1's binary_logloss: 0.304699
[300]	training's auc: 0.940398	training's binary_logloss: 0.266032	valid_1's auc: 0.925962	valid_1's binary_logloss: 0.305202
[350]	training's auc: 0.940584	training's binary_logloss: 0.265621	valid_1's auc: 0.92606	valid_1's binary_logloss: 0.305533
[400]	training's auc: 0.940725	training's binary_logloss: 0.2

LGBMClassifier(n_estimators=1000, n_jobs=4, num_leaves=3, random_state=4242,
               reg_alpha=10, reg_lambda=100, subsample=1)

In [157]:
# композиция предпоследнего слоя нн и lgbm:
def pipe_predict(user_input, item_input):
  return boosting.predict_proba(model.last_layer(user_input, item_input).detach().numpy().reshape(-1,32))[:,1]

In [158]:
# пары пользователь-фильм для теста
test_user_movie_set = set(zip(test_ratings['userId'], test_ratings['movieId']))

# все фильмы, с которыми взаимодействовали пользователи
user_interacted_movies = ratings.groupby('userId')['movieId'].apply(list).to_dict()


In [172]:
# так как в метрике используется случайный выбор 100 фильмов для HR@10,
# используем monte-carlo для хоть немного статистически-значимого результата
def hit_ratio(ismodel=True):
  hits = []
  for (u,i) in tqdm(test_user_movie_set):
    interacted_movies = user_interacted_movies[u]
    not_interacted_movies = set(all_movies) - set(interacted_movies)
    selected_not_interacted = list(np.random.choice(list(not_interacted_movies), 99))
    test_movies = selected_not_interacted + [i]
    
    if ismodel:
      predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_movies)).detach().numpy())
    else:
      predicted_labels = np.squeeze(pipe_predict(torch.tensor([u]*100), 
                                        torch.tensor(test_movies)))
    
    top10_movies = [test_movies[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    
    if i in top10_movies:
      hits.append(1)
    else:
      hits.append(0)
  return np.average(hits)

In [ ]:
# monte - carlo simulations
MC = 30
hr_model=[]
hr_boost=[]
for i in tqdm(range(MC)):
  hr_model.append(hit_ratio(ismodel=True))
  hr_boost.append(hit_ratio(ismodel=False))

In [193]:
print(f'init model mean HR = {sum(hr_model)/MC}, boost model mean HR = {sum(hr_boost)/MC}')
print(f'init std = {np.std(hr_model)},\t boost std = {np.std(hr_boost)}')

init model mean HR = 0.7801190476190475, boost model mean HR = 0.7781523809523809
init std = 0.0021444702922789748,	 boost std = 0.0021160010201935883


$\Rightarrow$ бустинг не дал ощутимого улучшения модели 